<a href="https://colab.research.google.com/github/prometricas/Abeba_Datos_Masivos/blob/main/personaGastosSinTarjetaCredito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Instalación de Java y Spark**

## **Instalar JDK de Java**

In [ ]:
!apt-get install -y openjdk-11-jdk-headless -qq > /dev/null

## **Instalar Spark**

In [ ]:
# Descargar Spark
!wget https://dlcdn.apache.org/spark/spark-3.5.7/spark-3.5.7-bin-hadoop3.tgz

--2026-01-12 16:25:15--  https://dlcdn.apache.org/spark/spark-3.5.7/spark-3.5.7-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400914067 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.7-bin-hadoop3.tgz’

spark-3.5.7-bin-had 100%[===================>] 382.34M   125MB/s    in 3.2s    

2026-01-12 16:25:34 (119 MB/s) - ‘spark-3.5.7-bin-hadoop3.tgz’ saved [400914067/400914067]



In [ ]:
# Descomprimir
!tar xf spark-3.5.7-bin-hadoop3.tgz

# Mostrar folder descomprimido
!ls /content

sample_data  spark-3.5.7-bin-hadoop3  spark-3.5.7-bin-hadoop3.tgz


## **Definir variables de entorno**


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.7-bin-hadoop3"

## **Instalar e Iniciar findspark**
Esto permite usar *PySpark* como una librería estándar en Python.

In [ ]:
!pip install findspark
import findspark
findspark.init()

# **`2. Importar bases de datos`**

In [ ]:
# Importar archivo
from google.colab import files
files.upload()

# Verificar
#!ls -lah
#!head -n 5 resultados_futbol.csv

Saving casoDePrueba1.txt to casoDePrueba1 (1).txt
Saving casoDePrueba3.txt to casoDePrueba3.txt


{'casoDePrueba1 (1).txt': b'Alice;Tarjeta de cr\xc3\xa9dito;100\nAlice;Tarjeta de cr\xc3\xa9dito;150\nAlice;Bizum;200\nBob;Tarjeta de cr\xc3\xa9dito;201\nLuis;Bizum;300\n',
 'casoDePrueba3.txt': b'Alice;Tarjeta de cr\xc3\xa9dito;1000\nAlice;Tarjeta de cr\xc3\xa9dito;1800\nAlice;Tarjeta de cr\xc3\xa9dito;2100\nBob;Bizum;2000\nAlice;Bizum;1000\nBob;Tarjeta de cr\xc3\xa9dito;1100\n'}

# **3. Crear script**

* Esta solución busca eficiencia evitando transformaciones innecesarias:
* En vez de filtrar líneas con tarjeta de crédito y luego “recuperar” personas con 0 deuda, mapea compras con tarjeta a 0 y hace *un único reduceByKey*.
* Así, una persona que solo pagó con tarjeta igualmente queda con suma 0: como “Bob;0” del ejemplo del enunciado.


In [ ]:
%%writefile personaGastosSinTarjetaCredito.py
import sys
from operator import add
from pyspark import SparkContext

def _to_pair(linea: str):
    # Leo y separo los 3 campos esperados: persona;método_pago;dinero_gastado
    persona, metodo, gasto = linea.split(";")
    persona = persona.strip()
    metodo_norm = metodo.strip().lower()

    # Convierto el gasto a número
    gasto_num = float(gasto.strip())

    # Sumo solo lo que NO es tarjeta de crédito; si es TDC, aporto 0
    if metodo_norm == "tarjeta de crédito":
        gasto_num = 0.0

    return (persona, gasto_num)

def _fmt_num(x: float) -> str:
    # Formateo sin .0 cuando el valor es entero
    if x.is_integer():
        return str(int(x))
    s = f"{x:.10f}".rstrip("0").rstrip(".")
    return s

def main():
    # Valido parámetros: entrada y salida
    if len(sys.argv) != 3:
        print("Uso: personaGastosSinTarjetaCredito.py <ruta_entrada> <ruta_salida>", file=sys.stderr)
        sys.exit(1)

    ruta_entrada = sys.argv[1]
    ruta_salida = sys.argv[2]

    sc = SparkContext.getOrCreate()
    sc.setLogLevel("WARN")

    # Cargo datos, transformo a (persona, gasto_sin_TDC), agrego por clave y genero salida persona;gasto
    rdd = sc.textFile(ruta_entrada)
    resultado = (
        rdd
        .map(_to_pair)
        .reduceByKey(add)
        .map(lambda kv: f"{kv[0]};{_fmt_num(kv[1])}")
    )

    resultado.saveAsTextFile(ruta_salida)
    sc.stop()

if __name__ == "__main__":
    main()

Writing personaGastosSinTarjetaCredito.py


# **4. Ejecutar con Spark**

## **Caso 1**

In [ ]:
# Limpio la carpeta de salida por si existiera de una ejecución previa
!rm -rf resultado_caso1

# Ejecutar con spark-submit
!$SPARK_HOME/bin/spark-submit personaGastosSinTarjetaCredito.py casoDePrueba1.txt resultado_caso1

26/01/12 16:57:22 INFO SparkContext: Running Spark version 3.5.7
26/01/12 16:57:22 INFO SparkContext: OS info Linux, 6.6.105+, amd64
26/01/12 16:57:22 INFO SparkContext: Java version 17.0.17
26/01/12 16:57:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/12 16:57:23 INFO ResourceUtils: ==============================================================
26/01/12 16:57:23 INFO ResourceUtils: No custom resources configured for spark.driver.
26/01/12 16:57:23 INFO ResourceUtils: ==============================================================
26/01/12 16:57:23 INFO SparkContext: Submitted application: personaGastosSinTarjetaCredito.py
26/01/12 16:57:23 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: M

In [ ]:
# Verificación de resultados
!cat resultado_caso1/part-*

Alice;200
Bob;0
Luis;300
